In [ ]:
!rm -rf /content/drive


Connect Google Drive to access input videos and save outputs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

for root, dirs, files in os.walk("/content/drive/MyDrive"):
    if "PlayerTracking" in dirs:
        print("Found at:", os.path.join(root, "PlayerTracking"))
        break

Found at: /content/drive/MyDrive/PlayerTracking


In [ ]:
import os

print(os.listdir("/content/drive/MyDrive/PlayerTracking"))


['12081088_3840_2160_60fps.mp4', '18884230-hd_1920_1080_60fps.mp4', '14770541_3840_2160_60fps.mp4', '5837595-uhd_3840_2160_24fps.mp4', '12059268_3840_2160_60fps.mp4', '14770543_3840_2160_60fps.mp4', '19216260-hd_1920_1080_30fps.mp4', '20366524-uhd_3840_2160_30fps.mp4', 'frames_football1', 'frames', 'video_720p.mp4']


In [ ]:
path = "/content/drive/MyDrive/PlayerTracking"

if os.path.exists(path):
    print("✅ Folder exists:", path)
else:
    print("❌ Folder NOT found")


✅ Folder exists: /content/drive/MyDrive/PlayerTracking


In [ ]:
import os, glob

BASE = "/content/drive/MyDrive/PlayerTracking"
videos = sorted(glob.glob(os.path.join(BASE, "*.mp4")))

print("Found videos:", len(videos))
for v in videos:
    print(" -", os.path.basename(v))


Found videos: 9
 - 12059268_3840_2160_60fps.mp4
 - 12081088_3840_2160_60fps.mp4
 - 14770541_3840_2160_60fps.mp4
 - 14770543_3840_2160_60fps.mp4
 - 18884230-hd_1920_1080_60fps.mp4
 - 19216260-hd_1920_1080_30fps.mp4
 - 20366524-uhd_3840_2160_30fps.mp4
 - 5837595-uhd_3840_2160_24fps.mp4
 - video_720p.mp4


Install YOLOv8 and OpenCV

In [ ]:
!pip -q install ultralytics opencv-python

from ultralytics import YOLO
print("✅ Setup OK")


✅ Setup OK


# **Player** Detection (YOLO) + Save Output Video

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")   # fast model

video_path = videos[0]       # first video
print("Using:", video_path)

model.predict(
    source=video_path,
    conf=0.4,
    save=True
)


Using: /content/drive/MyDrive/PlayerTracking/12059268_3840_2160_60fps.mp4

WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/294) /content/drive/MyDrive/PlayerTracking/12059268_3840_2160_60fps.mp4: 384x640 8 persons, 1123.7ms
video 1/1 (frame 2/294) /content/drive/MyDrive/PlayerTracking/12059268_3840_2160_60fps.mp4: 384x640 8 persons, 326.8ms
video 1/1 (frame 3/294) /content/drive/MyDrive/PlayerTracking/12059268_3840_2160_60fps.mp4: 384x640 8 persons, 341.9ms
video 1/1 (frame 4/294) /c

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p

checks whether the detection results were saved

In [ ]:
import glob

print(glob.glob("runs/detect/*"))

['runs/detect/predict']


In [ ]:
print(glob.glob("runs/detect/predict*"))

['runs/detect/predict']


In [ ]:
import os

print(os.listdir("runs/detect/predict"))

['12059268_3840_2160_60fps.avi']


displays the generated detection result video

In [ ]:
from IPython.display import Video

Video("runs/detect/predict/12059268_3840_2160_60fps.avi", embed=True)


Buffered data was truncated after reaching the output size limit.

Resize the video before displaying

In [ ]:
!ffmpeg -y -i "runs/detect/predict/12059268_3840_2160_60fps.avi" \
  -vf "scale=1280:720" \
  -vcodec libx264 -crf 28 \
  "runs/detect/predict/detect_small.mp4"


In [ ]:
from IPython.display import Video
Video("runs/detect/predict/detect_small.mp4", embed=True)


# **Player Tracking Using ByteTrack - IDs**

In [ ]:
model.track(
    source=video_path,
    conf=0.4,
    save=True,
    tracker="bytetrack.yaml"
)


Reload Video Path

In [ ]:
import glob
import os

BASE = "/content/drive/MyDrive/PlayerTracking"
videos = sorted(glob.glob(os.path.join(BASE, "*.mp4")))

video_path = videos[0]   # choose first video
print("Using:", video_path)


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

model.track(
    source=video_path,
    conf=0.4,
    save=True,
    tracker="bytetrack.yaml"
)



In [ ]:
import glob
print("Track folders:", glob.glob("runs/track/*"))
print("All track videos:", glob.glob("runs/track/**/*.*", recursive=True))


In [ ]:
import glob, os
track_vids = glob.glob("runs/track/**/*.avi", recursive=True) + glob.glob("runs/track/**/*.mp4", recursive=True)
print(track_vids)

One time Setup

Mount Google Drive + Install Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip -q install ultralytics opencv-python ffmpeg-python


In [ ]:
import os, glob

BASE = "/content/drive/MyDrive/PlayerTracking"
videos = sorted(glob.glob(os.path.join(BASE, "*.mp4")))

print("Found videos:", len(videos))
for v in videos[:10]:
    print(" -", os.path.basename(v))

video_path = videos[0]
print("\n✅ Using:", video_path)


Selecting the Video from Google Drive

In [ ]:
import os, glob

BASE = "/content/drive/MyDrive/PlayerTracking"
videos = sorted(glob.glob(os.path.join(BASE, "*.mp4")))

video_path = videos[0]
print("✅ Using:", video_path)



YOLO Player Detection

In [ ]:
from ultralytics import YOLO

det_model = YOLO("yolov8n.pt")

det_model.predict(
    source=video_path,
    conf=0.4,
    save=True,
    project="runs",
    name="detect_run"
)


Check Where YOLO Saved Result

In [ ]:
import os

print("Folders inside runs:")
print(os.listdir("runs"))




In [ ]:
import glob

print("All detect folders:")
print(glob.glob("runs/*"))


In [ ]:
import glob
print(glob.glob("runs/detect/*"))


In [ ]:
import os
print(os.listdir(glob.glob("runs/detect/predict*")[0]))


In [ ]:
import glob
print(glob.glob("runs/detect/predict*/*.avi"))
print(glob.glob("runs/detect/predict*/*.mp4"))


Set video_path (run once)

In [ ]:
import os, glob

BASE = "/content/drive/MyDrive/PlayerTracking"
videos = sorted(glob.glob(os.path.join(BASE, "*.mp4")))
print("Found:", len(videos), "videos")

video_path = videos[0]
print("✅ Using:", video_path)


In [ ]:
small_vid = "/content/drive/MyDrive/PlayerTracking/video_720p.mp4"

!ffmpeg -y -i "{video_path}" -vf "scale=1280:720" -r 30 -vcodec libx264 -crf 28 "{small_vid}"

import os
print("Small video exists?", os.path.exists(small_vid))


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

model.track(
    source=small_vid,
    conf=0.4,
    save=True,
    tracker="bytetrack.yaml",
    project="runs",
    name="track_run",
    verbose=True
)


In [ ]:
import os
print("Track_run exists?", os.path.exists("runs/track_run"))


In [ ]:
import glob

track_dirs = glob.glob("runs/**/track_run*", recursive=True)
print("Found track dirs:", track_dirs)


In [ ]:
track_dir = track_dirs[-1]
track_vids = glob.glob(track_dir + "/*.avi") + glob.glob(track_dir + "/*.mp4")
print(track_vids)

track_out = track_vids[0]
print("✅ Using tracking output:", track_out)



In [ ]:
!ffmpeg -y -i "{track_out}" -vf "scale=1280:720" -vcodec libx264 -crf 28 "{track_dir}/tracking_small.mp4"


In [ ]:
from IPython.display import Video
Video(f"{track_dir}/tracking_small.mp4", embed=True)


# **Pose / Keypoint Detection**

In [ ]:
from ultralytics import YOLO

pose_model = YOLO("yolov8n-pose.pt")

pose_model.predict(
    source=small_vid,
    conf=0.3,
    save=True,
    project="runs",
    name="pose_run",
    verbose=True
)


Locate Pose Output Folder and Video

In [ ]:
import glob, os

pose_dirs = sorted(glob.glob("runs/**/pose_run*", recursive=True))
print("Pose dirs found:", pose_dirs)

pose_dir = pose_dirs[-1]   # use the latest one
print("✅ Using pose_dir:", pose_dir)

pose_vids = glob.glob(pose_dir + "/*.avi") + glob.glob(pose_dir + "/*.mp4")
print("Pose videos:", pose_vids)


In [ ]:
pose_out = pose_vids[0]
print("✅ Pose output:", pose_out)

!ffmpeg -y -i "{pose_out}" -vf "scale=1280:720" -vcodec libx264 -crf 28 "{pose_dir}/pose_small.mp4"

print("✅ Made:", f"{pose_dir}/pose_small.mp4")


Display Pose Result

In [ ]:
from IPython.display import Video
Video("runs/pose/runs/pose_run/pose_small.mp4", embed=True)


In [ ]:
import os

print(os.listdir("/content"))





In [ ]:
print(os.listdir("/content/runs") if os.path.exists("/content/runs") else "No runs folder")
